# Training school "Data and Models" INRAE 2024

## Exercise 1: Introduction to ML - Regression
In this first exercise, we are going to load a data set for regression, and try a few algorithms on it. The data set contains measurements performed on different white wines, along with an evaluation of quality. The objective is to predict the value of the quality, starting from the values of the other variables.

In [ ]:
# install the openml library; openml is a large collection of openly accessible datasets
!pip install openml

# load the dataset
import openml

dataset = openml.datasets.get_dataset(44971)
#dataset = openml.datasets.get_dataset(560) # don't worry about this commented line, for the moment

print(dataset)

Let's take a closer look at the data. The data is organized in a way similar to an Excel spreadsheet, with the rows corresponding to the samples (instances of measurements), while the columns correspond to the features (the measured variables) and the target (the variable we are interested in predicting).

Here below 'X' contains all values of the features; 'y' are the values of the target;

In [ ]:
# get the data from the dataset as a dataframe, an object that makes it easy to manipulate
# datasets; for example, dataframes allow you to index columns by name, remove non-valid values
# transform categorical features into numerical features, etc.
df, *_ = dataset.get_data()
print(df)

# as you noticed, some of the columns contain strings instead of numbers; we call these
# "categorical" columns or features. We need to change that, as most ML algorithms
# only process numbers. Don't worry too much about this part, it's just replacing strings
# with numbers
categorical_columns = df.select_dtypes(include=['category', 'object', 'string'])
for c in categorical_columns :
  df[c].replace({category : index for index, category in enumerate(df[c].astype('category').cat.categories)}, inplace=True)

# also, remove all rows that contain invalid numerical values (for example, missing values)
df.dropna(inplace=True)

# name of the target column
target_feature = dataset.default_target_attribute
other_features = [c for c in df.columns if c != target_feature]

# get just the data without the column headers, as numerical matrices
X = df[other_features].values
y = df[target_feature].values

print("X=", X)

In [ ]:
# setup a machine learning algorithm
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()

# train the algorithm on the data
print("Training regressor \"%s\"..." % regressor.__class__.__str__)
regressor.fit(X, y)

# obtain the predictions of the algorithm
y_predicted = regressor.predict(X)

# compare the predictions to the measured values, using an error metric
from sklearn.metrics import mean_squared_error, r2_score
print("Mean squared error: %.4f" % mean_squared_error(y, y_predicted))
print("R2: %.4f" % r2_score(y, y_predicted))

The Mean Squared Error (MSE) can be harder to interpret, as its values depends on the original values of the target variable:

\begin{equation}
MSE = \frac{ \sum\limits_{i=0}^N{(y_i - \hat{y}_i)^2} }{ N }
\end{equation}

where $y_i$ is the measured value of the target variable for sample $i$, $\hat{y}_i$ is the prediction of the trained model for sample $i$, and $N$ is the number of samples. To make it more interpretable, we could either normalize the target variable (but this brings more issues, as we will see later), or use $R2$ (the coefficient of correlation):

\begin{equation}
R2 = 1 - \frac{ \sum\limits_{i=0}^N{(y_i - \hat{y}_i)^2} }{ \sum\limits_{i=0}^N{(y_i - \bar{y})^2} }
\end{equation}

where $\bar{y}$ is the mean value of $y$ over the available $N$ samples. In general, $R2$ is easier to interpret as values close to $1.0$ show that the model performs well, and values close to $0.0$ (or negative) show that the model is garbage.

Now, the $R2$ of the model is around $0.85$, so it looks good. But is it _really_? The issue here is that we tested the model on the same data it was trained on, so the model probably _overfits_ the training data, showing a performance that is better than what it would do on unseen data. Let's try out this hypothesis by randomly splitting the data into two parts, a 'training' set and 'test' set. The 'test' set will be unseen during training.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42) # we use 10% of the data for the test, the rest for the training

regressor.fit(X_train, y_train)
y_train_predicted = regressor.predict(X_train)
y_test_predicted = regressor.predict(X_test)

print("R2 on train: %.4f" % r2_score(y_train, y_train_predicted))
print("R2 on test: %.4f" % r2_score(y_test, y_test_predicted))

The result we obtained is quite different! But maybe we were just unlucky, since the split in the data was random, we accidentally created a situation where the algorithm performed badly. There is a way to obtain a more reliable evaluation of quality: repeat the train/test split several times, and observe the mean performance (and its standard deviation) on all the test data.

This process is commonly known as _cross-validation_. In this case, we are going to use a _k-fold cross-validation_, which just means we are going to split the data into _k_ groups (called _folds_), and for _k_ times, we are going to train the algorithm on _k-1_ folds and test it on the remaining one. We selected _k=5_ to speed up the computation, but a more common choice is _k=10_. This might take a while.

In [ ]:
from sklearn.model_selection import cross_validate
results = cross_validate(regressor, X, y, cv=5, scoring='r2', return_train_score=True)

# cross_validate returns a dictionary of results, with keys corresponding to arrays of results
train_performance = results["train_score"]
test_performance = results["test_score"]

# let's compute mean and standard deviation of the scores
import numpy as np # library that performs quick mathematical operations
print("Mean R2 score on training set: %.4f (std=%.4f)" % (np.mean(train_performance), np.std(train_performance)))
print("Mean R2 score on test set: %.4f (std=%.4f)" % (np.mean(test_performance), np.std(test_performance)))


So, it was not an accident: the problem is actually quite hard, and even if the ML algorithm is able to learn patterns in the training set, they do not generalize well on unseen data.

We can even observe the pattern visually, creating a "predicted vs. measured" plot for all samples when they are evaluated in test. Ideally, we would like all points in the plot to be as close as possible to the red dotted line in the middle (which corresponds to a perfect prediction).

In [ ]:
from sklearn.metrics import PredictionErrorDisplay
from sklearn.model_selection import cross_val_predict

# get all predictions for the test samples
y_pred = cross_val_predict(regressor, X, y)

# create the plot
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
display = PredictionErrorDisplay.from_predictions(
        y_true=y,
        y_pred=y_pred,
        kind="actual_vs_predicted",
        ax=ax,
        scatter_kwargs={"alpha": 0.2, "color": "tab:blue"},
        line_kwargs={"color": "tab:red"},
    )
ax.set_title("Predicted vs measured")

Well, that was a bit disappointing. However, as the final step of this exercise, we are now going to go back to the very first code cell in this notebook, uncomment the commented line (thus selecting another dataset to work on) and RERUN EVERYTHING, without changing anything else.

One of the big advantages of ML is the possibility of running the same algorithm on multiple different datasets, without the need of performing ad-hoc modifications. This is true only for common tasks such as regression and classification, but a considerable number of interesting real-world problems falls into these categories.